In [1]:
import feedparser
import requests
from datetime import datetime

# Fonction pour convertir une date Feedparser en ISO 8601
def convert_to_iso8601(date_str):
    try:
        # Parse la date avec le format de Feedparser
        parsed_date = datetime.strptime(date_str, "%a, %d %b %Y %H:%M:%S %z")
        # Convertit en format ISO 8601
        return parsed_date.isoformat()
    except ValueError:
        return None  # Retourne None si la date est invalide ou absente


# Liste des flux RSS (Feedly et Google Alerts)
rss_feeds = [
    "https://www.grafikart.fr/",  # lien feedly 
    "https://www.google.com/alerts/feeds/00304130937566327789/2520171459693073861",     # Google Alert
]

# Notion API Configuration
NOTION_API_URL = "https://api.notion.com/v1/pages"
NOTION_API_KEY = "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"  # Remplace par ton Token Notion
DATABASE_ID = "https://www.notion.so/144359de043f80359769f4296f5cef97?v=4abb8d8ad9914cfd979fe28f38e70d95&pvs=4"  # Remplace par l'ID de ta base de données

# Fonction pour analyser les flux RSS
def parse_rss_feeds(feeds):
    articles = []
    for feed_url in feeds:
        print(f"Analyse du flux RSS : {feed_url}")
        feed = feedparser.parse(feed_url)
        
        # Vérifie si le flux est valide
        if 'title' in feed.feed:
            print(f"Flux trouvé : {feed.feed.title}")
        else:
            print("Erreur : Flux non valide.")
            continue
        
        for entry in feed.entries:
            article = {
                "title": entry.title,
                "summary": entry.summary if "summary" in entry else "N/A",
                "link": entry.link,
                "published": entry.published if "published" in entry else "N/A"            
            }
            articles.append(article)
    
    return articles

# Fonction pour envoyer un article à Notion
def add_article_to_notion(article):
    headers = {
        "Authorization": f"Bearer {NOTION_API_KEY}",
        "Content-Type": "application/json",
        "Notion-Version": "2022-06-28"
    }
    # Conversion de la date
    iso_date = convert_to_iso8601(article["published"])

    data = {
        "parent": {"database_id": DATABASE_ID},
        "properties": {
            "Title": {"title": [{"text": {"content": article["title"]}}]},
            "Summary": {"rich_text": [{"text": {"content": article["summary"]}}]},
            "Link": {"url": article["link"]},
            "Published": {"date": {"start": iso_date} if iso_date else None}       
        }
    }
    response = requests.post(NOTION_API_URL, headers=headers, json=data)
    if response.status_code == 200:
        print(f"Article ajouté à Notion : {article['title']}")
    else:
        print(f"Erreur lors de l'ajout à Notion : {response.status_code}, {response.text}")


# Fonction principale
if __name__ == "__main__":
    # Étape 1 : Analyse des flux RSS
    collected_articles = parse_rss_feeds(rss_feeds)
    
    # Étape 3 : Envoi à Notion
    print(f"\nAjout des articles à Notion...")
    for article in collected_articles:
        add_article_to_notion(article)

    print(f"\n{len(collected_articles)} articles ajoutés à Notion.")

Analyse du flux RSS : https://www.grafikart.fr/
Erreur : Flux non valide.
Analyse du flux RSS : https://www.google.com/alerts/feeds/00304130937566327789/2520171459693073861
Flux trouvé : Google Alert - Cyber sécurité

Ajout des articles à Notion...
Erreur lors de l'ajout à Notion : 400, {"object":"error","status":400,"code":"validation_error","message":"body failed validation. Fix one:\nbody.parent.page_id should be defined, instead was `undefined`.\nbody.parent.database_id should be a valid uuid, instead was `\"https://www.notion.so/144359de043f80359769f4296f5cef97...`.","request_id":"9a7ef8a2-b5ce-4a31-8a75-d7752fd9e4cb"}
Erreur lors de l'ajout à Notion : 400, {"object":"error","status":400,"code":"validation_error","message":"body failed validation. Fix one:\nbody.parent.page_id should be defined, instead was `undefined`.\nbody.parent.database_id should be a valid uuid, instead was `\"https://www.notion.so/144359de043f80359769f4296f5cef97...`.","request_id":"d4243c21-15c2-4a99-8787-